# YOLOv8 Ensemble

## Libraries

In [1]:
from ultralytics import YOLO
import torch
import torchvision

print(torch.cuda.is_available())
print(torchvision.__version__)

True
0.17.0+cu118


## Hyperparameter Tuning

In [ ]:
model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n.pt')

model.tune(data='crystals_2600.yaml',
           patience=3,
           epochs=25,
           iterations=100,
           plots=False, 
           save=False, 
           val=False
           )

## Generate YOLOv8 Models
- Optimizer defaults to AdamW

In [2]:
num_models = 3
# model = YOLO('yolov8s-p2.yaml').load('yolov8s.pt')

for i in range(num_models):

    model = YOLO('yolov8n.yaml').load('yolov8n.pt')

    results = model.train(data='crystals_2600.yaml', 
                        epochs=100,
                        patience=10,
                        imgsz=608,
                        project='models',
                        name=f'yolov8n_{i+1}',
                        exist_ok = True,
                        seed = i,
                        deterministic = False)
    
    del model
    del results
    torch.cuda.empty_cache()

Transferred 355/355 items from pretrained weights
Ultralytics YOLOv8.1.29 🚀 Python-3.11.8 torch-2.2.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=crystals_2600.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=608, save=True, save_period=-1, cache=False, device=None, workers=8, project=models, name=yolov8n_1, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=False, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf

train: Scanning C:\GitHub\mt2024-YOLOv8-ensemble\datasets\crystals_2600\labels\train.cache... 2080 images, 11 backgrounds, 0 corrupt: 100%|██████████| 2080/2080 [00:00<?, ?it/s]
val: Scanning C:\GitHub\mt2024-YOLOv8-ensemble\datasets\crystals_2600\labels\val.cache... 520 images, 3 backgrounds, 0 corrupt: 100%|██████████| 520/520 [00:00<?, ?it/s]


Plotting labels to models\yolov8n_1\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 608 train, 608 val
Using 8 dataloader workers
Logging results to models\yolov8n_1
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      2.06G      1.488      2.998      1.525         73        608: 100%|██████████| 130/130 [00:16<00:00,  7.88it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:03<00:00,  5.09it/s]

                   all        520       1404        0.4      0.354       0.31        0.2


...

      2/100      2.07G      1.463      2.318      1.474         60        608: 100%|██████████| 130/130 [00:13<00:00,  9.39it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:03<00:00,  5.44it/s]

                   all        520       1404       0.44      0.338      0.328      0.208



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      3/100      2.07G      1.513      2.232      1.519         74        608:  10%|█         | 13/130 [00:01<00:14,  8.14it/s]


KeyboardInterrupt: 

## Evaluate
- P (Precision): The accuracy of the detected objects, indicating how many detections were correct.
- R (Recall): The ability of the model to identify all instances of objects in the images.


## Predict

In [ ]:
model = YOLO("models\yolov8n_1\weights\\best.pt")
images = (["CVAT\images\\02ke_D8_ImagerDefaults_9.jpg", 
           "CVAT\images\\01dd_D12_ImagerDefaults_9.jpg",
           "CVAT\images\\038f_B2_ImagerDefaults_9.jpg"
           ]) 

# Run batched inference on a list of images
results = model(images, save_txt = True, save_conf = True, project = "single", exist_ok = True)

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    result.show()  # display to screen
    # result.save(filename='result.jpg')  # save to disk

## Ensemble
Ensemble output is average of the probabilities.

### Load Models

In [ ]:
images = (["CVAT\images\\02ke_D8_ImagerDefaults_9.jpg", 
           "CVAT\images\\01dd_D12_ImagerDefaults_9.jpg",
           "CVAT\images\\038f_B2_ImagerDefaults_9.jpg"
           ]) 

# 
model_1 = YOLO("models\yolov8n_1\weights\\best.pt")
results_1 = model_1(images, save_txt = True, save_conf = True, project = "ensemble", name = "model_1", exist_ok = True)

for result in results_1:
    boxes = result.boxes  # Boxes object for bounding box outputs
    result.show()  # display to screen
    # result.save(filename='result.jpg')  # save to disk

del model_1
del results_1
torch.cuda.empty_cache()

# 
model_2 = YOLO("models\yolov8n_2\weights\\best.pt")
results_2 = model_2(images, save_txt = True, save_conf = True, project = "ensemble", name = "model_2", exist_ok = True)

for result in results_2:
    boxes = result.boxes  # Boxes object for bounding box outputs
    result.show()  # display to screen
    # result.save(filename='result.jpg')  # save to disk

del model_2
del results_2
torch.cuda.empty_cache()

# 
model_3 = YOLO("models\yolov8n_3\weights\\best.pt")
results_3 = model_3(images, save_txt = True, save_conf = True, project = "ensemble", name = "model_3", exist_ok = True)

for result in results_3:
    boxes = result.boxes  # Boxes object for bounding box outputs
    result.show()  # display to screen
    # result.save(filename='result.jpg')  # save to disk

del model_3
del results_3
torch.cuda.empty_cache()
